In [4]:
import sys
import os
import numpy as np
import pandas as pd
from FCLayer import FcLayer
from ACLayer import AcLayer
from Network import Network
from activation_functions import sigmoid, relu, sigmoid_prime, tanh, tanh_prime,relu_prime
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import pickle


In [5]:
data=pd.read_csv('spam_final_df.data')
data.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [6]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.2, random_state=26)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
learning_rate=0.1

In [7]:
# X_test=np.expand_dims(X_test,axis=1)
# X_train=np.expand_dims(X_train,axis=1)
# y_train=np.expand_dims(y_train,axis=1)
# y_test=np.expand_dims(y_test,axis=1)
network = Network(learning_rate, epochs=1000, node_counts = [3, 3, 3])


In [8]:
# input_values = 57
# output_values = 1
# intermediate_values = 3
# layers = 3
# epochs = 1000

# for j in range(layers):
#     i = intermediate_values
#     o = intermediate_values

#     if not j:
#         i = input_values
#     elif j + 1 == layers:
#         o = output_values

#     fc = FcLayer(i, o)
#     ac = AcLayer(tanh, tanh_prime)
#     network.add(fc)
#     network.add(ac)
network.fit(X_train, y_train)

out = network.predict(X_test)

In [9]:
y_pred=np.where(np.array(out) > 0.5, 1, 0)
np.unique(y_pred)

array([0, 1])

In [10]:
def score(y_true, y_pred):
    correct_predictions = 0
    total_predictions = len(y_true)

    for true_label, predicted_label in zip(y_true, y_pred):
        if true_label == predicted_label:
            correct_predictions += 1



    accuracy = correct_predictions / total_predictions
    return accuracy
accuracy=score(y_test,y_pred)
print("accuracy")
print(accuracy)

accuracy
0.9294245385450597


In [11]:
print(classification_report(y_pred=np.squeeze(y_pred),y_true=np.squeeze(y_test)))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94       553
           1       0.91      0.92      0.91       368

    accuracy                           0.93       921
   macro avg       0.93      0.93      0.93       921
weighted avg       0.93      0.93      0.93       921



In [12]:

X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.2, random_state=26)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train.shape

(3680,)

In [13]:
def create_model(hidden_units, dropout_rate):
    model = Sequential([
        tf.keras.layers.Flatten(input_shape=(57,)),
        Dense(hidden_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(2, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
model = KerasClassifier(build_fn=create_model)

param_grid = {
    'hidden_units': [64, 128, 256],
    'dropout_rate': [0.2, 0.3, 0.4],
    'batch_size': [32, 64],
    'epochs': [100]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Train the model using cross-validation
grid_search_result = grid_search.fit(X_train, y_train)

Epoch 1/100


C:\Users\vali\AppData\Local\Temp\ipykernel_816\2291812154.py:10: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model)


77/77 [==============================] - 0s 829us/step - loss: 0.4584 - accuracy: 0.8002
Epoch 2/100
77/77 [==============================] - 0s 737us/step - loss: 0.2835 - accuracy: 0.9066
Epoch 3/100
77/77 [==============================] - 0s 724us/step - loss: 0.2325 - accuracy: 0.9201
Epoch 4/100
77/77 [==============================] - 0s 737us/step - loss: 0.2076 - accuracy: 0.9303
Epoch 5/100
77/77 [==============================] - 0s 737us/step - loss: 0.1933 - accuracy: 0.9364
Epoch 6/100
77/77 [==============================] - 0s 737us/step - loss: 0.1802 - accuracy: 0.9425
Epoch 7/100
77/77 [==============================] - 0s 737us/step - loss: 0.1744 - accuracy: 0.9413
Epoch 8/100
77/77 [==============================] - 0s 737us/step - loss: 0.1696 - accuracy: 0.9405
Epoch 9/100
77/77 [==============================] - 0s 737us/step - loss: 0.1573 - accuracy: 0.9466
Epoch 10/100
77/77 [==============================] - 0s 737us/step - loss: 0.1493 - accuracy: 0.9466
E

In [14]:
#print(grid_search_result)
with open("grid_search_spam.pickle", "wb") as pickle_file:
    pickle.dump(grid_search_result, pickle_file)

In [15]:
# grid_search_result

# GridSearchCV(cv=3,
#              estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x00000206A5C3DA10>,
#              param_grid={'batch_size': [32, 64],
#                          'dropout_rate': [0.2, 0.3, 0.4], 'epochs': [100],
#                          'hidden_units': [64, 128, 256]})